#### Simulating the travelling salesman problem with three cities.

* Importing important libraries

In [2]:
from qiskit import *
from qiskit.circuit import parameter
from qiskit.circuit import ParameterVector, Parameter
from qiskit.visualization import plot_histogram
from IPython.core.display import Latex
from qiskit_algorithms.optimizers import ADAM, COBYLA
from qiskit_algorithms.gradients import SPSAEstimatorGradient
from qiskit_aer import AerSimulator, StatevectorSimulator
import numpy as np
import ot
import re
from qiskit.circuit.library.standard_gates import RXGate, RYGate, RZGate

* Cost Matrix

In [5]:
# Tsp-4
K = np.array([[0.        ,0.54194679,0.58401706,0.19804715],
 [0.54194679,0.        ,0.77658693,0.43439787],
 [0.58401706,0.77658693,0.        ,0.73731612],
 [0.19804715,0.43439787,0.73731612,0.        ]])
K = K.flatten()

In [3]:
# TSP- 3

# K = np.array([[0.        ,0.81721768,0.55177632],
#  [0.81721768,0.        ,0.6720918 ],
#  [0.55177632,0.6720918 ,0.        ]])

# K = K.flatten()

In [15]:
# n = 4
# beta_para = 5.11401041
# theta1 = [-0.2132781]
# theta2 = [0.98435579]
# alpha = [3.00780862]

* Parameters

In [4]:
n = 4    # Number of cities
beta_para = (2 * np.pi * np.random.rand(1))[0]         # Beta parameters
theta1 = (2 * np.pi * np.random.rand(1))        # Perceptron layers 1st parameters
theta2 =(2 * np.pi * np.random.rand(1))   # Perceptron layers 2nd parameters
alpha = (2 * np.pi * np.random.rand(1))       # Alpha parameters for pooling layer

# initial_point = np.array([beta_para for i in range(n*n)]+[theta1[0] for i in range(n*n)]+
#                          [theta2[0] for i in range(n*n)]+[alpha[0] for i in range(n*n)])


In [8]:
class TSP_Solver:
    def __init__(self, reference_circuit, variational_circuit,MCRX, n, K, theta1, theta2, alpha, matching_matrix, loss):
        self.ref_cir = reference_circuit
        self.var_cir = variational_circuit
        self.MCRX = MCRX
        self.n = n
        self.n1 = n*n  # Squared of number of cities
        self.K = K
        self.alpha = alpha
        self.theta1 = theta1
        self.theta2 = theta2
        self.matching_matrix = matching_matrix  # A constraint matrix for matching with the ground truth
        self.loss = loss
    
    "Creating an object of encoding layer"
    def encoding_layer(self):
        for q in range(len(self.K)):
            self.ref_cir.ry(self.K[q], q)
        self.ref_cir.barrier()
    
    'Function that return the ctrl qubits for constrained layer'
    def ctrl_qubits(self, matrix, row_idx, col_idx):
        neighbors = []
        num_rows = len(matrix)
        num_cols = len(matrix[0])

        # Add neighbors from the same row
        for j in range(num_cols):
            if j != col_idx:  # Exclude the element itself
                neighbors.append(matrix[row_idx][j])

        # Add neighbors from the same column
        for i in range(num_rows):
            if i != row_idx:  # Exclude the element itself
                neighbors.append(matrix[i][col_idx])

        return neighbors
            
    "Defining contraint layer"
    def constraint_layer(self): 
        matrix = np.array([i for i in range(self.n*self.n)]).reshape(self.n, self.n)
        for i in range(self.n):
            for j in range(self.n):  
                ctrl = list(self.ctrl_qubits(matrix, i, j))+[matrix[i][j]]
                self.var_cir.append(self.MCRX, ctrl)
                
        self.var_cir.barrier()
    
    "Perceptron layer"
    def perceptron_layer(self):
        rz = RZGate(self.theta1[0])
        ry = RYGate(self.theta2[0])
        for q in range(self.n1):
            self.var_cir.append(rz, [q])
            self.var_cir.append(ry, [q])
        self.var_cir.barrier()
               
    'Pooling Layer'
    def pooling_layer(self):
        ry = RYGate(self.alpha[0])
        for q in range(self.n1):
            self.var_cir.append(ry, [q])
    
    'composing the reference and variational circuits'
    # The circuit can be visualized as well from here.
    def ansatz(self):
        ansat = self.ref_cir.compose(self.var_cir)
        ansat.measure_all()
        return ansat
    
    'Statvector simulating'
    def statevect(self):
        simulator = StatevectorSimulator()
        result= simulator.run(transpile(self.ansatz(), simulator), shots = 1000).result()
        statevector = result.get_statevector()
        return statevector.draw('latex')
    
    'Reshaping result to matrix'
    def x_matrix(self):
        latex_string = self.statevect().data     # Stores the statevector as a latex string5
        latex_list = list(latex_string)        # This object stores the statevector as a list
        
        'Extracting only the states of the qubits'
        a = latex_list.index('|')
        b = latex_list.index('\\')
        y = latex_list[a+1:b]
        y1 = np.array([int(i) for i in y])
        Y1 = y1.reshape(self.n, self.n)  # Reshaping the measured states of qubits to a matrix 
        return Y1
        
#         # Define the size of marginals
#         n = Y1.shape[0]
#         m = Y1.shape[1]
        
#         # Define the cost matrix with the matching constraint
#         cost_matrix = np.multiply(Y1, self.matching_matrix) # Element-wise multiplication to zero out non-matching elements

#         # Apply Sinkhorn algorithm to converge to a doubly stochastic matrix with the matching constraint
#         X_ds = ot.sinkhorn(np.ones(n) / n, np.ones(m) / m, cost_matrix, reg=0.01)
#         return X_ds
    
    def cost_fun(self):
        """Cost function of circuit parameters on training data.
            The optimizer will attempt to minimize this."""
        X = list(self.matching_matrix.flatten())
        Y = list(self.x_matrix().flatten())
        for i in range(len(X)):
            self.loss += (-X[i]*np.log(Y[i]) + (1-X[i])*np.log(1-Y[i]))
        return self.loss


#### Calling the whole class here
* Number of Cities 'n'
* Quantum circuit 
* Applying Hadammard to all qubits

In [10]:
def call_pass_obj_fun(TSP):
    TSP.encoding_layer()
    TSP.constraint_layer()
    TSP.perceptron_layer()
    TSP.pooling_layer()
    TSP.ansatz()
    TSP.statevect()
    TSP.x_matrix()
    TSP.cost_fun()
    return TSP.loss
    
    
# Define the matching constraint
m_c = np.array([[1 ,0.,0., 0.],
                 [0.,0 ,1,0.],
                 [0.,1,0,0.],
                 [0, 0.,0., 1]])


reference_circuit = QuantumCircuit(n*n)      # Circuit to which data be encoded
reference_circuit.h([i for i in range(n*n)])     
variational_circuit = QuantumCircuit(n*n)     # Circuit for

loss = 0
#Preparing the multi-controlled X rotational gat
MCRX=RXGate(beta_para).control(6, ctrl_state='000000')

TSP = TSP_Solver(reference_circuit, variational_circuit, MCRX, n, K, theta1, theta2, alpha, m_c, loss)

##### Objective function for optimizer

In [7]:
def objective_function(params):

    theta1_values = params[9:18]  # Extract first 9 values for theta1
    theta2_values = params[18:27]  # Extract next 9 values for theta2
    alpha = params[27:36]  # Extract alpha
    beta_para = params[:1]  # Extract beta_para

    # Set theta1 and theta2 values
    TSP.theta1 = theta1_values
    TSP.theta2 = theta2_values
    TSP.alpha = alpha

    # Update the beta_para for MCRX gate
    beta_angle = beta_para[0]  # Assuming you want to use the first value from beta_para
    MCRX = RXGate(beta_angle).control(6, ctrl_state='000000')
    TSP.MCRX = MCRX
    
    # Run the quantum neural network and compute loss
    obj_loss = call_pass_obj_fun(TSP)
    return obj_loss

##### Gradient for the optimizer


In [8]:
def gradient_function(params):

    epsilon = 1e-6  # pert value

    initial_loss = objective_function(params)

    params_plus = params.copy()+2*epsilon
    
    loss_plus = objective_function(params_plus)
    
    gradient = (loss_plus - initial_loss) / 2*epsilon

    return gradient

In [9]:

initial_point = np.array([beta_para for i in range(n*n)]+[theta1[0] for i in range(n*n)]+
                         [theta2[0] for i in range(n*n)]+[alpha[0] for i in range(n*n)])-

# Instantiate Adam optimizer
adam_optimizer = ADAM(maxiter= 2, tol=1e-06, lr=0.5, beta_1=0.9,
                      beta_2=0.99, noise_factor=1e-08, eps=1e-10, amsgrad=False, snapshot_dir=None)

# Run optimization
result = adam_optimizer.minimize(objective_function, initial_point, gradient_function)
result.x

array([ 5.11401041,  5.11401041,  5.11401041,  5.11401041,  5.11401041,
        5.11401041,  5.11401041,  5.11401041,  5.11401041,  5.11401041,
        5.11401041,  5.11401041,  5.11401041,  5.11401041,  5.11401041,
        5.11401041, -0.2132781 , -0.2132781 , -0.2132781 , -0.2132781 ,
       -0.2132781 , -0.2132781 , -0.2132781 , -0.2132781 , -0.2132781 ,
       -0.2132781 , -0.2132781 , -0.2132781 , -0.2132781 , -0.2132781 ,
       -0.2132781 , -0.2132781 ,  0.98435579,  0.98435579,  0.98435579,
        0.98435579,  0.98435579,  0.98435579,  0.98435579,  0.98435579,
        0.98435579,  0.98435579,  0.98435579,  0.98435579,  0.98435579,
        0.98435579,  0.98435579,  0.98435579,  3.00780862,  3.00780862,
        3.00780862,  3.00780862,  3.00780862,  3.00780862,  3.00780862,
        3.00780862,  3.00780862,  3.00780862,  3.00780862,  3.00780862,
        3.00780862,  3.00780862,  3.00780862,  3.00780862])

###### A main function from where the whole class or TSP-Simulation be controlled

In [113]:
!git add .
!git commit -m "Your commit message"

[main a2ff8a4] Your commit message
 9 files changed, 1811 insertions(+), 478 deletions(-)
 create mode 100644 .ipynb_checkpoints/Training TSP-Solver-checkpoint.ipynb
 create mode 100644 .ipynb_checkpoints/Untitled-checkpoint.ipynb
 create mode 100644 Training TSP-Solver.ipynb
 create mode 100644 Untitled.ipynb


In [114]:
!git push -u origin main

To https://github.com/Sajjad-Ahmad-phy/Final-Year-Project.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/Sajjad-Ahmad-phy/Final-Year-Project.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
